# Assignment: 1 - Camera Calibration 

### Importing necessary libraries

In [82]:
import numpy as np
import pandas as pd
from scipy import linalg

### Normalization

In [83]:
def Normalization(nd, x):
    #CONVERTING TO NUMPY ARRAY
    x = np.asarray(x)

    #CALCULATING MEAN AND STANDARD DEVIATION
    m, s = np.mean(x, 0), np.std(x)

    print("mean = ", m, "std = ", s)

    #NORMALIZATION MATRIX FOR WORLD POINTS(3D) AND IMAGE-PIXEL POINTS(2D)
    if nd == 2:
        Tr = np.array([[s, 0, m[0]], [0, s, m[1]], [0, 0, 1]])
    else:
        Tr = np.array([[s, 0, 0, m[0]], [0, s, 0, m[1]], [0, 0, s, m[2]], [0, 0, 0, 1]])
        
    Tr = np.linalg.inv(Tr)
    x = np.dot( Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x

### DLT Camera Calibration

In [84]:
def DLTcalib(nd, xyz, img_pt_):

    if (nd != 3):
        raise ValueError('%dD DLT unsupported.' %(nd))
    
    # Converting all variables to numpy array
    xyz = np.asarray(xyz)
    img_pt_ = np.asarray(img_pt_)

    n = xyz.shape[0]

    print("\n\nNUMBER OF POINTS: {}\n\n".format(n))
    # Validating the parameters:
    if img_pt_.shape[0] != n:
        raise ValueError('Object (%d points) and image (%d points) have different number of points.' %(n, img_pt_.shape[0]))

    if (xyz.shape[1] != 3):
        raise ValueError('Incorrect number of coordinates (%d) for %dD DLT (it should be %d).' %(xyz.shape[1],nd,nd))

    if (n < 6):
        raise ValueError('%dD DLT requires at least %d calibration points. Only %d points were entered.' %(nd, 2*nd, n))
        
    # Normalize the data to improve the DLT quality (DLT is dependent of the system of coordinates).
    # This is relevant when there is a considerable perspective distortion.
    # Normalization: mean position at origin and mean distance equals to 1 at each direction.
    Txyz, xyzn = Norm(nd, xyz)
    Timg_pt_, img_pt_n = Norm(2, img_pt_)

    A = []

    for i in range(n):
        x, y, z = xyzn[i, 0], xyzn[i, 1], xyzn[i, 2]
        u, v = img_pt_n[i, 0], img_pt_n[i, 1]
        A.append( [x, y, z, 1, 0, 0, 0, 0, -u * x, -u * y, -u * z, -u] )
        A.append( [0, 0, 0, 0, x, y, z, 1, -v * x, -v * y, -v * z, -v] )

    # Convert A to array
    A = np.asarray(A) 

    # Find the 11 parameters:
    U, S, V = np.linalg.svd(A)

    # The parameters are in the last line of Vh and normalize them
    L = V[-1, :] / V[-1, -1]

    # Camera projection matrix
    H = L.reshape(3, nd + 1)


    # Denormalization
    H = np.dot( np.dot( np.linalg.pinv(Timg_pt_), H ), Txyz )
    H = H / H[-1, -1]
    L = H.flatten()

    # Mean error of the DLT (mean residual of the DLT transformation in units of camera coordinates):
    img_pt_2 = np.dot( H, np.concatenate( (xyz.T, np.ones((1, xyz.shape[0]))) ) ) 
    img_pt_2 = img_pt_2 / img_pt_2[2, :] 

    # Mean distance:
    projected_pts = img_pt_2.T
    actual_pixels = img_pt_

    # Printing teh actual and the estimated points
    print('\n\nACTUAL PTS   |             ESTIMATED PTS')
    for i in range(len(img_pt_2.T)):
        print("{}   <---->  {}".format(actual_pixels[i], projected_pts[i]))

    err = np.sqrt( np.mean(np.sum( (img_pt_2[0:2, :].T - img_pt_)**2, 1)) ) 

    return L, err

In [85]:
def get_intrest_points():

    # WORLD CORDINATES
    data = pd.read_excel("dataset.xlsx")
    x = np.array(data['X'])
    y = np.array(data['Y'])
    z = np.array(data['Z'])

    # IMAGE PIXEL COORDINATES
    u = np.array(data['x'])
    v= np.array(data['y'])

    # LIST OF WORLD PTS AND CORRESPONDING PIXEL PTS
    world_pts = []
    pixel_pts = []

    for i in range(10):
        world_pts.append([x[i], y[i], z[i]])
        pixel_pts.append([u[i], v[i]])

    print("DATA FRAME")
    print(data)

    # from sklearn.utils import shuffle
    # world_pts, pixel_pts = shuffle(world_pts, pixel_pts, random_state = 0)

    print("\n\nWORLD points")
    print(world_pts)
    return world_pts, pixel_pts
   

### RQ Decomposition 

In [86]:
def RQ_decomposition(P):
    M = P[0:3,0:3]
    print("\nM:\n",M)
    K, R = linalg.rq(M)
    T = np.diag(np.sign(np.diag(K)))
    print(T)

    K = np.dot(K, T)
    R = np.dot(T, R)
    C = np.dot(linalg.inv(-M), P[:, 3])
    return K, R, C


In [87]:
    def camera_param(P):
        print("\nP:\n",P)

        K, R, C = RQ_decomposition(P)

        print("\n\n K MATRIX: ")
        print(K)
        print("\n\n R MATRIX: ")
        print(R)
        print("\n\n Camera center: ")
        print(C)

        print("\n\n Normalized camera matrix: ")
        print(K/K[2][2])


In [88]:
#camera_param(P)

In [89]:

xyz, img_pt_ = get_intrest_points()
nd = 3
P, err = DLTcalib(nd, xyz, img_pt_)
P = P.reshape(3,4)

DATA FRAME
       X     Y   Z    x     y  Unnamed: 5
0   20.0   0.0   0  708   991           1
1    0.0  20.0   0  213   998           2
2    0.0   0.0  20  455   597           3
3   20.0  20.0   0  464  1192           4
4   20.0   0.0  20  727   658           5
5    0.0  20.0  20  182   665           6
6    0.0  15.0  10  279   810           7
7    0.0   7.5  15  373   699           8
8   12.5  17.5   0  396  1082           9
9    5.0   0.0   5  516   828          10
10  10.0  10.0   0  463   998          11
11  10.0   0.0  10  576   779          12
12   0.0  10.0  10  343   784          13
13   0.0   0.0   0  463   874          14
14  15.0  10.0   0  529  1038          15
15  15.0   0.0  10  642   805          16
16  25.0   0.0  25  829   576          17
17   0.0  25.0  25   76   586          18


WORLD points
[[20.0, 0.0, 0], [0.0, 20.0, 0], [0.0, 0.0, 20], [20.0, 20.0, 0], [20.0, 0.0, 20], [0.0, 20.0, 20], [0.0, 15.0, 10], [0.0, 7.5, 15], [12.5, 17.5, 0], [5.0, 0.0, 5]]


NUMBER OF

### Projection matrix (P)

In [90]:
print('Matrix')
print(P)

Matrix
[[ 5.71644451e+00 -1.44566549e+01 -2.00255961e+00  4.62669395e+02]
 [-3.29192351e+00 -3.34205262e+00 -1.59251192e+01  8.71183066e+02]
 [-9.31402693e-03 -9.72493565e-03 -3.73705201e-03  1.00000000e+00]]


### RMSE

In [91]:
print('\nError')
print(err)


Error
1.7889239466599147


### Camera Parameters

In [92]:
camera_param(P.reshape(3,4))


P:
 [[ 5.71644451e+00 -1.44566549e+01 -2.00255961e+00  4.62669395e+02]
 [-3.29192351e+00 -3.34205262e+00 -1.59251192e+01  8.71183066e+02]
 [-9.31402693e-03 -9.72493565e-03 -3.73705201e-03  1.00000000e+00]]

M:
 [[ 5.71644451e+00 -1.44566549e+01 -2.00255961e+00]
 [-3.29192351e+00 -3.34205262e+00 -1.59251192e+01]
 [-9.31402693e-03 -9.72493565e-03 -3.73705201e-03]]
[[ 1.  0.  0.]
 [ 0. -1.  0.]
 [ 0.  0.  1.]]


 K MATRIX: 
[[1.41286161e+01 1.29046670e-01 6.78589769e+00]
 [0.00000000e+00 1.40909532e+01 8.77841314e+00]
 [0.00000000e+00 0.00000000e+00 1.39746567e-02]]


 R MATRIX: 
[[ 0.72305535 -0.69077545 -0.00449824]
 [ 0.18159435  0.19635489 -0.96357057]
 [-0.66649415 -0.695898   -0.26741637]]


 Camera center: 
[45.84792805 45.18150475 35.74580636]


 Normalized camera matrix: 
[[1.01101705e+03 9.23433566e+00 4.85586005e+02]
 [0.00000000e+00 1.00832196e+03 6.28166642e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [93]:
import cv2 as cv

In [94]:
img = cv.imread('THREE_PLANE_DATA/image (12).jpg')
cv.imshow('orginal image', img)

In [95]:
def Norm(nd, x):
    #CONVERTING TO NUMPY ARRAY
    x = np.asarray(x)
#print(x)

    #CALCULATING MEAN AND STANDARD DEVIATION
    m = np.mean(x, 0)

    #NORMALIZATION MATRIX FOR WORLD POINTS(3D) AND IMAGE-PIXEL POINTS(2D)
    if nd == 2:
        A = np.array([[1, 0, -m[0]], [0, 1, -m[1]], [0, 0, 1]])
        dist = np.mean(np.sqrt(np.sum(np.square(x-m))))
        s2D = np.sqrt(2)/dist
        print("mean = ", m)
        print("scale = ", s2D)

        S1 = np.array([[s2D, 0, 0], [0, s2D, 0], [0, 0, 1]])
        Tr = np.matmul(S1, A)

    else:
        dist = np.mean(np.sqrt(np.sum(np.square(x-m))))
        s3D = np.sqrt(3)/dist
        Tr = np.diag([s3D, s3D, s3D, 1])
        Tr[0:3, 3] = -m*s3D
        
    #Tr = np.linalg.inv(Tr)
    x = np.dot(Tr, np.concatenate( (x.T, np.ones((1,x.shape[0]))) ) )
    x = x[0:nd, :].T

    return Tr, x

In [96]:
Normalization(3, xyz)

mean =  [ 7.75 10.    9.  ] std =  8.819564741086841


(array([[ 0.11338428,  0.        ,  0.        , -0.87872817],
        [ 0.        ,  0.11338428,  0.        , -1.1338428 ],
        [ 0.        ,  0.        ,  0.11338428, -1.02045852],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 1.38895743, -1.1338428 , -1.02045852],
        [-0.87872817,  1.1338428 , -1.02045852],
        [-0.87872817, -1.1338428 ,  1.24722708],
        [ 1.38895743,  1.1338428 , -1.02045852],
        [ 1.38895743, -1.1338428 ,  1.24722708],
        [-0.87872817,  1.1338428 ,  1.24722708],
        [-0.87872817,  0.5669214 ,  0.11338428],
        [-0.87872817, -0.2834607 ,  0.68030568],
        [ 0.53857533,  0.8503821 , -1.02045852],
        [-0.31180677, -1.1338428 , -0.45353712]]))

In [97]:
Norm(3, xyz)

(array([[ 0.03605213,  0.        ,  0.        , -0.27940403],
        [ 0.        ,  0.03605213,  0.        , -0.36052133],
        [ 0.        ,  0.        ,  0.03605213, -0.3244692 ],
        [ 0.        ,  0.        ,  0.        ,  1.        ]]),
 array([[ 0.44163863, -0.36052133, -0.3244692 ],
        [-0.27940403,  0.36052133, -0.3244692 ],
        [-0.27940403, -0.36052133,  0.39657346],
        [ 0.44163863,  0.36052133, -0.3244692 ],
        [ 0.44163863, -0.36052133,  0.39657346],
        [-0.27940403,  0.36052133,  0.39657346],
        [-0.27940403,  0.18026067,  0.03605213],
        [-0.27940403, -0.09013033,  0.2163128 ],
        [ 0.17124763,  0.270391  , -0.3244692 ],
        [-0.09914337, -0.36052133, -0.14420853]]))